In [37]:
import os
import torch
from torchvision.datasets.utils import download_url
import pandas as pd

### Load data

In [38]:
# Read data - short version 3400+ documents
reuters = pd.read_pickle('input/reuters_small.pkl')
print(len(reuters))
reuters[0:5]

3426


,codes,headline,text,classes,classes_pad
0,"[C18, C181, CCAT]",Eureko is latest suitor for French insurer GAN.,"\nEureko, an alliance of six European financia...","[25, 26, 44]","[25, 26, 44, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,"[G15, GCAT]",Reuter EC Report Long-Term Diary for July 28 -...,\n****\nHIGHLIGHTS\n****\nLUXEMBOURG - Luxembo...,"[80, 90]","[80, 90, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"
2,"[G15, GCAT]",Official Journal contents - OJ L 190 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]","[80, 90, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"
3,"[G15, GCAT]",Official Journal contents - OJ C 221 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]","[80, 90, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"
4,"[G15, GCAT]",Official Journal contents - OJ C 220 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]","[80, 90, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"


In [39]:
# load full data, if produced. 435 MB. NOT in Github currently. 
# You can produce this in about 30 min with preprocess_data.ipynb

#reuters = pd.read_pickle('input/reuters_all.pkl')

In [40]:
# read classcodes
classcodes= pd.read_csv('input/classcodes.csv')
print(len(classcodes))
#classcodes[0:12]

126


In [41]:
# add index field to DataFrame
classcodes = classcodes.reset_index()
# Create dictionary index/int to classcode and classcode to int
itocode = dict(zip(classcodes.index, classcodes.Code))
codetoi = dict(zip(classcodes.Code, classcodes.index))
def listToInt(mylist):
    return [codetoi[item] for item in mylist]

reuters['codes'] = [listToInt(codelist) for codelist in reuters.codes]
reuters[0:3]
# Multihot, for single list - one row
def multihot(tags):
    return [1 if tag in tags else 0 for tag in taglist]

# list of classes, 126 int: [0...125]
taglist = list(classcodes.index)
Y_hot = [multihot(claslist) for claslist in reuters.codes]
reuters['codes'] = Y_hot

In [43]:
reuters[0:5]

,codes,headline,text,classes,classes_pad
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Eureko is latest suitor for French insurer GAN.,"\nEureko, an alliance of six European financia...","[25, 26, 44]","[25, 26, 44, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Reuter EC Report Long-Term Diary for July 28 -...,\n****\nHIGHLIGHTS\n****\nLUXEMBOURG - Luxembo...,"[80, 90]","[80, 90, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Official Journal contents - OJ L 190 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]","[80, 90, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Official Journal contents - OJ C 221 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]","[80, 90, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Official Journal contents - OJ C 220 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]","[80, 90, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"


### Using DataSet


Example in:
https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/A%20-%20Using%20TorchText%20with%20Your%20Own%20Datasets.ipynb

In [114]:
from torchtext import data
from torchtext import datasets

#Define the Fields
TEXT = data.Field()
HEADLINE = data.Field()

LABELS = data.LabelField(sequential=False, use_vocab=False)
#CLASSES = data.LabelField()

In [115]:
reuters[0:5]

,codes,headline,text,classes,classes_pad
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Eureko is latest suitor for French insurer GAN.,"\nEureko, an alliance of six European financia...","[25, 26, 44]","[25, 26, 44, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Reuter EC Report Long-Term Diary for July 28 -...,\n****\nHIGHLIGHTS\n****\nLUXEMBOURG - Luxembo...,"[80, 90]","[80, 90, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Official Journal contents - OJ L 190 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]","[80, 90, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Official Journal contents - OJ C 221 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]","[80, 90, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Official Journal contents - OJ C 220 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]","[80, 90, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"


In [116]:
len(reuters)

3426

In [117]:
### Test, save pd df to JSON
                            # one record at time, one record by line
reuters.to_json('input/reuters_small.json', orient='records', lines=True)

In [118]:
# split it
train = reuters[0:2500]
test = reuters[2500:3000]
val = reuters[3000:len(reuters)]

train.to_json('input/train.json', orient='records', lines=True)
test.to_json('input/test.json', orient='records', lines=True)
val.to_json('input/val.json', orient='records', lines=True)

In [119]:
# Tell torchText which Fields to apply to which json elements

fields = {'headline': ('h', HEADLINE), 'text': ('t', TEXT), 'codes': ('l', LABELS)}

In [120]:
fields

{'headline': ('h', <torchtext.data.field.Field at 0x7fe01a3c5588>),
 'text': ('t', <torchtext.data.field.Field at 0x7fe01a3c5748>),
 'codes': ('l', <torchtext.data.field.LabelField at 0x7fe01a3c59b0>)}

In [121]:
# Create dataset (TabularDataset)
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = 'input',
                                        train = 'train.json',
                                        validation = 'val.json',
                                        test = 'test.json',
                                        format = 'json',
                                        fields = fields
)

In [122]:
# test
print(vars(train_data[2]))

{'h': ['Official', 'Journal', 'contents', '-', 'OJ', 'L', '190', 'of', 'July', '19,', '1997.'], 't': ['*', '(Note', '-', 'contents', 'are', 'displayed', 'in', 'reverse', 'order', 'to', 'that', 'in', 'the', 'printed', 'Journal)', '*', 'COMMISSION', 'DECISION', 'of', '16', 'July', '1997', 'amending', 'Decision', '97/285/EC', 'concerning', 'certain', 'protection', 'measures', 'relating', 'to', 'classical', 'swine', 'fever', 'in', 'Spain', '(Text', 'with', 'EEA', 'relevance)', 'COMMISSION', 'REGULATION', '(EC)', 'No', '1399/97', 'of', '18', 'July', '1997', 'establishing', 'the', 'standard', 'import', 'values', 'for', 'determining', 'the', 'entry', 'price', 'of', 'certain', 'fruit', 'and', 'vegetables', 'COMMISSION', 'REGULATION', '(EC)', 'No', '1398/97', 'of', '18', 'July', '1997', 'fixing', 'the', 'agricultural', 'conversion', 'rates', 'COMMISSION', 'REGULATION', '(EC)', 'No', '1397/97', 'of', '18', 'July', '1997', 'fixing,', 'for', 'the', '1996/97', 'marketing', 'year,', 'the', 'specific

### continue

device = torch.device('cpu')
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train, val, test), batch_sizes=(16, 256, 256),
    sort_key=lambda x: len(x.text), device=device)


In [123]:
TEXT.build_vocab(train)
HEADLINE.build_vocab(train)
LABELS.build_vocab(train)

In [124]:
TEXT.vocab

In [125]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 2500
Number of validation examples: 426
Number of testing examples: 500


In [126]:
TEXT.build_vocab(train_data, max_size=25000)
LABELS.build_vocab(train_data)

In [127]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABELS vocabulary: {len(LABELS.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABELS vocabulary: 2


In [128]:
print(TEXT.vocab.freqs.most_common(10))

[('the', 22529), ('of', 12791), ('to', 12004), ('and', 9792), ('in', 9599), ('a', 8831), ('on', 5491), ('said', 4417), ('for', 4282), ('The', 3677)]


In [129]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', 'of', 'to', 'and', 'in', 'a', 'on', 'said']


In [130]:
print(LABELS.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fe01ccc8730>, {0: 0, 1: 1})


In [131]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device,
    sort_key= lambda x: len(x.t)
)

### simple model

In [146]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):

        #x = [sent len, batch size]
        
        embedded = self.embedding(x)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [147]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 256
OUTPUT_DIM = 126

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [148]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [149]:
criterion = nn.BCEWithLogitsLoss()

In [150]:
model = model.to(device)
criterion = criterion.to(device)

In [151]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

### F1 accuracy

The results will be ranked according to the highest micro-averaged F1 score. 
This will be calculated using the f1_score function found in scikit-learn, using a command like 
f1_score(y_true, y_pred, average='micro') where y_true is the matrix with the ground truth, and y_pred 
the predicted output. Both matrices are binary, a 1 in row i and column j means that the image/document
i contains the label j.

Scikit:  Micro-average in F1-score
 
'micro':
    Calculate metrics globally by counting the total true positives, false negatives and false positives.

In [152]:
# f1 score for BATCH
from sklearn.metrics import f1_score
def f1_accuracy(preds, y):
    """
    Returns f1 accuracy from sklearn
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    
    preds_cpu = rounded_preds.cpu().data.numpy()
    y_cpu = y.cpu().data.numpy()
    f1 = f1_score(y_cpu, preds_cpu, average='micro')
    return f1 

In [157]:
import numpy as np
def f1_own_accuracy(preds, y):
    '''Returns counts of true_pos, false_pos and false_negative.
    For counting precision, recall and F1 globally
    precision = true_pos / (true_pos + false_pos)
    recall = true_pos / (true_pos + false_neg)
    '''

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    #correct = (rounded_preds == y).float() #convert into float for division 
    
    preds = rounded_preds.cpu().data.numpy()
    y = y.cpu().data.numpy()
        
    # True positive
    tpos = np.sum(np.logical_and(preds == 1, y == 1))
 
    # True negative
    #tneg = np.sum(np.logical_and(preds == 0, y == 0))
 
    # False positive
    fpos = np.sum(np.logical_and(preds == 1, y == 0))
 
    # False negative
    fneg = np.sum(np.logical_and(preds == 0, y == 1))

    return tpos, fpos, fneg

In [154]:
# F1 version
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    #epoch_acc = 0
    
    epoch_tpos = 0
    epoch_fpos = 0
    epoch_fneg = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.t).squeeze(1)
        
        loss = criterion(predictions, batch.l.float())
        
        tpos, fpos, fneg = f1_own_accuracy(predictions, batch.l.float())
        epoch_tpos += tpos
        epoch_fpos += fpos
        epoch_fneg += fneg
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        #epoch_acc += acc.item()
        
    epoch_precision = epoch_tpos / (epoch_tpos + epoch_fpos)
    epoch_recall = epoch_tpos / (epoch_tpos + epoch_fneg)
    epoch_f1 = 2* (  (epoch_precision * epoch_recall) / (epoch_precision + epoch_recall))
    
    return epoch_loss / len(iterator), epoch_precision, epoch_recall, epoch_f1

In [155]:
# F1 version
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    #epoch_acc = 0
    epoch_tpos = 0
    epoch_fpos = 0
    epoch_fneg = 0    
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.t).squeeze(1)
            
            loss = criterion(predictions, batch.l.float())
            
            tpos, fpos, fneg = f1_own_accuracy(predictions, batch.l.float())
            epoch_tpos += tpos
            epoch_fpos += fpos
            epoch_fneg += fneg            

            epoch_loss += loss.item()
            #epoch_acc += acc.item()
            
    epoch_precision = epoch_tpos / (epoch_tpos + epoch_fpos)
    epoch_recall = epoch_tpos / (epoch_tpos + epoch_fneg)
    epoch_f1 = 2* (  (epoch_precision * epoch_recall) / (epoch_precision + epoch_recall))            
        
    return epoch_loss / len(iterator), epoch_precision, epoch_recall, epoch_f1

In [158]:
# Ver2 - F1
N_EPOCHS = 4

for epoch in range(N_EPOCHS):
    print(epoch)

    train_loss, train_precision, train_recall, train_f1 = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model, valid_iterator, criterion)
    
    #print(f'| Epoch:{epoch+1:02} | Train Loss: {train_loss:.3f} | Tr Precision: {train_precision:.3f} | Tr recall: {train_recall:.3f} | Tr f1: {train_f1:.3f} | Valid f1: {valid_f1:.3f} |')    
    print(f'| Ep:{epoch+1:02} |Tr Loss:{train_loss:.3f} |Prec:{train_precision:.3f} |Rec:{train_recall:.3f} |Tr f1:{train_f1:.3f} |Val prec:{valid_precision:.3f} |Val rec:{valid_recall:.3f} |Val f1:{valid_f1:.3f} |')        

0
| Ep:01 |Tr Loss:0.711 |Prec:0.027 |Rec:0.666 |Tr f1:0.053 |Val prec:0.026 |Val rec:0.662 |Val f1:0.050 |
1
| Ep:02 |Tr Loss:0.708 |Prec:0.028 |Rec:0.644 |Tr f1:0.053 |Val prec:0.026 |Val rec:0.643 |Val f1:0.051 |
2
| Ep:03 |Tr Loss:0.705 |Prec:0.028 |Rec:0.629 |Tr f1:0.053 |Val prec:0.027 |Val rec:0.636 |Val f1:0.051 |
3
| Ep:04 |Tr Loss:0.702 |Prec:0.028 |Rec:0.623 |Tr f1:0.054 |Val prec:0.027 |Val rec:0.624 |Val f1:0.051 |


In [ ]:
# ...120 epochs here

In [164]:
# Ver2 - F1
N_EPOCHS = 20
for epoch in range(N_EPOCHS):
    print(epoch)

    train_loss, train_precision, train_recall, train_f1 = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model, valid_iterator, criterion)
    
    #print(f'| Epoch:{epoch+1:02} | Train Loss: {train_loss:.3f} | Tr Precision: {train_precision:.3f} | Tr recall: {train_recall:.3f} | Tr f1: {train_f1:.3f} | Valid f1: {valid_f1:.3f} |')    
    print(f'| Ep:{epoch+1:02} |Tr Loss:{train_loss:.3f} |Prec:{train_precision:.3f} |Rec:{train_recall:.3f} |Tr f1:{train_f1:.3f} |Val prec:{valid_precision:.3f} |Val rec:{valid_recall:.3f} |Val f1:{valid_f1:.3f} |')        

0
| Ep:01 |Tr Loss:0.285 |Prec:0.390 |Rec:0.159 |Tr f1:0.225 |Val prec:0.283 |Val rec:0.163 |Val f1:0.207 |
1
| Ep:02 |Tr Loss:0.282 |Prec:0.401 |Rec:0.159 |Tr f1:0.227 |Val prec:0.284 |Val rec:0.163 |Val f1:0.207 |
2
| Ep:03 |Tr Loss:0.280 |Prec:0.405 |Rec:0.158 |Tr f1:0.228 |Val prec:0.288 |Val rec:0.163 |Val f1:0.208 |
3
| Ep:04 |Tr Loss:0.278 |Prec:0.406 |Rec:0.158 |Tr f1:0.227 |Val prec:0.290 |Val rec:0.163 |Val f1:0.209 |
4
| Ep:05 |Tr Loss:0.275 |Prec:0.393 |Rec:0.158 |Tr f1:0.226 |Val prec:0.291 |Val rec:0.163 |Val f1:0.209 |
5
| Ep:06 |Tr Loss:0.272 |Prec:0.386 |Rec:0.158 |Tr f1:0.224 |Val prec:0.291 |Val rec:0.163 |Val f1:0.209 |
6
| Ep:07 |Tr Loss:0.271 |Prec:0.391 |Rec:0.158 |Tr f1:0.225 |Val prec:0.296 |Val rec:0.163 |Val f1:0.210 |
7
| Ep:08 |Tr Loss:0.270 |Prec:0.405 |Rec:0.158 |Tr f1:0.228 |Val prec:0.299 |Val rec:0.163 |Val f1:0.211 |
8
| Ep:09 |Tr Loss:0.267 |Prec:0.395 |Rec:0.157 |Tr f1:0.225 |Val prec:0.301 |Val rec:0.163 |Val f1:0.212 |
9
| Ep:10 |Tr Loss:0.264 |Pr

### OLD versions of train, evaluate and run. Not using global F1

In [71]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.t).squeeze(1)
        
        loss = criterion(predictions, batch.l.float())
        
        #acc = binary_accuracy(predictions, batch.l.float())
        acc = f1_accuracy(predictions, batch.l.float())
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [91]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.t).squeeze(1)
            
            loss = criterion(predictions, batch.l.float())
            
            #acc = binary_accuracy(predictions, batch.l.float())
            acc = f1_accuracy(predictions, batch.l.float())

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [75]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    print(epoch)

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Score: {train_acc*100:.2f} | Val. Loss: {valid_loss:.3f} | Val. Score: {valid_acc*100:.2f} |')    

0
| Epoch: 01 | Train Loss: 0.682 | Train Score: 5.49 | Val. Loss: 0.681 | Val. Score: 5.60 |
1
| Epoch: 02 | Train Loss: 0.679 | Train Score: 5.39 | Val. Loss: 0.678 | Val. Score: 5.76 |
2
| Epoch: 03 | Train Loss: 0.676 | Train Score: 5.66 | Val. Loss: 0.674 | Val. Score: 5.94 |
3
| Epoch: 04 | Train Loss: 0.673 | Train Score: 5.81 | Val. Loss: 0.671 | Val. Score: 6.14 |
4
| Epoch: 05 | Train Loss: 0.670 | Train Score: 6.00 | Val. Loss: 0.668 | Val. Score: 6.16 |
